In [17]:
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import MultiQueryRetriever
from langchain.chat_models import ChatOpenAI

In [2]:
loader = WikipediaLoader(query="MKUltra")

In [4]:
documents = loader.load()

/usr/local/Cellar/jupyterlab/4.2.3/libexec/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/Cellar/jupyterlab/4.2.3/libexec/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [6]:
splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)

In [7]:
chunked_article_docs=splitter.split_documents(documents)

Created a chunk of size 525, which is longer than the specified 500
Created a chunk of size 501, which is longer than the specified 500


In [11]:
embedding_function = OpenAIEmbeddings()

In [16]:
db = Chroma.from_documents(chunked_article_docs, embedding_function, persist_directory=".mkultra")
db.persist()

In [18]:
question = "When was this declassified?"

In [19]:
llm = ChatOpenAI()

In [20]:
retriever = MultiQueryRetriever.from_llm(retriever=db.as_retriever(), llm=llm)

In [21]:
import logging
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

In [25]:
print(retriever.get_relevant_documents(query=question)[0])

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the date of the declassification of this information?', '2. Can you provide the time when this was officially declassified?', '3. Do you know the specific moment when this document became unclassified?']


page_content='== Background ==
In 1974, a New York Times article was published that accused the CIA of illegal operations committed against US citizens. Authored by Seymour M. Hersh, it documented an intelligence operation against the anti-war movement, as well as "break-ins, wiretapping and the surreptitious inspection of mail" conducted since the 1950s. According to former CIA Official Cord Meyer, these disclosures "Convinced large sections of the American public that the CIA had become a domestic Gestapo and stimulated an overwhelming demand for the wide-ranging congressional investigations that were to follow."
Hersh had been tipped off to the possibility of an "in house operation" by an unidentified member of the CIA in spring of 1974. He embarked on an investigation, speaking to sources that included CIA Chief of Counterintelligence James Angleton. Although he was not aware of its existence, Hersh uncovered much information that had been documented in the "Family Jewels", a repor